## Notebook Overview: Daily Evolution Animations of Water Column Structure

This notebook creates animated visualizations (GIFs) of daily changes in water column structure, particularly temperature profiles and mixed layer depths (MLD), across dock locations. These animations help illustrate how stratification and mixing evolve over time during the observation period.

### Objectives:
- **Prepare and Interpolate Data**: Load CTD cast data and interpolate temperature profiles across depth and dock position.
- **Animate Temporal Evolution**: Generate daily time-step animations showing temperature gradients and MLD indicators.
- **Visual Encoding**: Use consistent color normalization, annotations, and legends to clearly convey vertical and horizontal structure changes.
- **Export GIFs**: Produce and save animated GIFs suitable for use in presentations, reports, or further analysis.

The animations serve as a visual tool to examine how thermal structure and mixing evolve across days and dock regions, supporting interpretation of short-term limnological dynamics. Each section runs independantly.

# Hovmoller GIF

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle
from matplotlib.lines import Line2D
import scipy.interpolate as si
from pathlib import Path
from matplotlib.colors import Normalize
from matplotlib.cm     import ScalarMappable          # ← NEW

# ---------------------------------------------------------------------
# 1.  LOAD DATA
# ---------------------------------------------------------------------
csv_path = '/content/drive/MyDrive/Field_Camp /correlation/dock_casts_with_mld_patched.csv'
#csv_path = '/content/drive/MyDrive/MLD/Final_Dock_Cast_Data.csv'
df = pd.read_csv(csv_path)

df['datetime_local'] = pd.to_datetime(df['Cast time (local)'])
df['date']           = df['datetime_local'].dt.date

# Auto‑detect temperature column
temp_col = next(c for c in df.columns if 'temp' in c.lower())

# Global colour scale (STATIC for the whole animation)
global_min = np.nanmin(df[temp_col])
global_max = np.nanmax(df[temp_col])
norm       = Normalize(vmin=global_min, vmax=global_max)
cmap       = plt.get_cmap('viridis')                     # ← FIXED palette

# ---------------------------------------------------------------------
# 2.  BUILD CAST TABLE
# ---------------------------------------------------------------------
cast_tbl = (
    df.groupby('Cast ID')
      .agg(max_depth      = ('Depth (Meter)', 'max'),
           MLD            = ('MLD',           'first'),
           dock           = ('Dock Location', 'first'),
           date           = ('date',          'first'),
           datetime_local = ('datetime_local','first'))
      .reset_index()
)
cast_tbl = cast_tbl[cast_tbl['max_depth'] >= 6.0]

# ---------------------------------------------------------------------
# 3.  INTERPOLATION
# ---------------------------------------------------------------------
docks      = ['Dock_2', 'Dock_1', 'Dock_3']
avg_depths = {'Dock_1': 8.36, 'Dock_2': 7.17, 'Dock_3': 9.26}
z_grid     = np.arange(0, 9.1, 0.1)

prof_dict = {d: [] for d in docks}
mld_dict  = {d: [] for d in docks}
time_idx  = {d: [] for d in docks}

for _, row in cast_tbl.sort_values('datetime_local').iterrows():
    cast_rows = df[df['Cast ID'] == row['Cast ID']]
    z = cast_rows['Depth (Meter)'].values
    T = cast_rows[temp_col].values

    if len(z) < 2:
        continue

    Ti = si.interp1d(z, T, bounds_error=False, fill_value=np.nan)(z_grid)
    dock = row['dock']
    if dock in prof_dict:
        prof_dict[dock].append(Ti)
        mld_dict[dock].append(row['MLD'])
        time_idx[dock].append(row['datetime_local'])

for d in docks:
    prof_dict[d] = np.array(prof_dict[d])
    mld_dict[d]  = np.array(mld_dict[d])
    time_idx[d]  = np.array(time_idx[d])

n_frames = min(len(prof_dict[d]) for d in docks)

# ---------------------------------------------------------------------
# 4.  FIGURE + GRID SPEC SETUP
# ---------------------------------------------------------------------
plt.ioff()
fig = plt.figure(figsize=(12, 7))

gs = fig.add_gridspec(2, 3, height_ratios=[0.15, 0.85], hspace=0.15)

title_ax = fig.add_subplot(gs[0, :])
title_ax.axis('off')

axes = [fig.add_subplot(gs[1, col]) for col in range(3)]

label_map = {'Dock_2': 'West', 'Dock_1': 'Central', 'Dock_3': 'East'}

# ---------------------------------------------------------------------
# 5.  INITIAL PLOTTING
# ---------------------------------------------------------------------
im_handles    = []
label_handles = []
ml_lines      = []

for ax, dock in zip(axes, docks):
    img = ax.imshow(
        prof_dict[dock][0:1, :].T,
        origin='upper',
        aspect='auto',
        extent=[0, 1, z_grid[-1], z_grid[0]],
        norm=norm,                                 # ← STATIC scale
        cmap=cmap                                  # ← STATIC colours
    )
    ax.set_title(label_map[dock], fontsize=14)
    ax.set_xlabel('')
    im_handles.append(img)

    mld0 = mld_dict[dock][0]
    line = ax.hlines(mld0, 0, 1, colors='black', linestyles='--', linewidth=1)
    ml_lines.append(line)

    top_text = ax.text(0.05, 0.93, '', transform=ax.transAxes,
                       fontsize=11, color='white', weight='bold')
    bot_text = ax.text(0.05, 0.85, '', transform=ax.transAxes,
                       fontsize=11, color='white', weight='bold')
    label_handles.append((top_text, bot_text))

axes[0].set_ylabel('Depth (m)')

# ----- STATIC colour‑bar built from a dummy ScalarMappable ------------
sm   = ScalarMappable(norm=norm, cmap=cmap)
sm.set_array([])
cbar = fig.colorbar(sm, ax=axes, label='Temperature (°C)')
# ---------------------------------------------------------------------

# ---------------------------------------------------------------------
# 6.  DYNAMIC TITLE & LEGEND
# ---------------------------------------------------------------------
initial_date = time_idx[docks[0]][0].date()
title_text = title_ax.text(
    0.50, 0.60,
    f'Vertical Temperature Profile at Each Dock — {initial_date:%d %b %Y}',
    ha='center', va='center',
    fontsize=18, fontweight='bold'
)

legend_line = Line2D([0], [0], color='black', linestyle='--',
                     linewidth=1, label='MLD')
title_ax.legend(handles=[legend_line], loc='center',
                bbox_to_anchor=(0.50, 0.25), frameon=False, fontsize=12)

# ---------------------------------------------------------------------
# 7.  UPDATE FUNCTION
# ---------------------------------------------------------------------
def update(frame):
    current_dt = time_idx[docks[0]][frame].date()
    title_text.set_text(
        f'Vertical Temperature Profile at Each Dock — {current_dt:%d %b %Y}'
    )

    for j, dock in enumerate(docks):
        temp_profile = prof_dict[dock][frame]
        mld          = mld_dict[dock][frame]
        img          = im_handles[j]
        line         = ml_lines[j]
        top_txt, bot_txt = label_handles[j]

        img.set_data(temp_profile[None, :].T)          # only pixels update
        line.set_segments([[(0, mld), (1, mld)]])

        top_mask = z_grid <= mld
        bot_mask = (z_grid > mld) & (z_grid <= avg_depths[dock])
        top_mean = np.nanmean(temp_profile[top_mask])
        bot_mean = np.nanmean(temp_profile[bot_mask])

        top_txt.set_text(f"MLD mean: {top_mean:.2f}°C")
        bot_txt.set_text(f"Below MLD: {bot_mean:.2f}°C")

    artists = [title_text] + im_handles + ml_lines
    for pair in label_handles:
        artists.extend(pair)
    return artists

ani = animation.FuncAnimation(
    fig, update, frames=n_frames, blit=True,
    interval=1000, repeat=False
)

# ---------------------------------------------------------------------
# 8.  EXPORT TO GIF
# ---------------------------------------------------------------------
gif_path = Path('/content/drive/MyDrive/Field_Camp /correlation/hovmoller_temp_with_mld_FINAL.gif')
ani.save(gif_path, writer='pillow', fps=1, dpi=150)
print(f'✅ GIF saved to: {gif_path}')

✅ GIF saved to: /content/drive/MyDrive/Field_Camp /correlation/hovmoller_temp_with_mld_FINAL.gif


# MLD GIF

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib.patches import Rectangle, Patch
import datetime as dt
from pathlib import Path
from matplotlib.lines import Line2D


# ---------------------------------------------------------------------
# 1.  LOAD & PREP
# ---------------------------------------------------------------------

csv_path = '/content/drive/MyDrive/Field_Camp /correlation/dock_casts_with_mld_patched.csv'
df = pd.read_csv(csv_path)

df['datetime_local'] = pd.to_datetime(df['Cast time (local)'])
df['date']           = df['datetime_local'].dt.date

cast_tbl = (
    df.groupby('Cast ID')
      .agg(
          max_depth=('Depth (Meter)', 'max'),
          MLD      =('MLD',          'first'),
          dock     =('Dock Location','first'),
          date     =('date',         'first')
      )
      .reset_index()
)
cast_tbl = cast_tbl[cast_tbl['max_depth'] >= 6.0]

mean_mld = (
    cast_tbl
      .groupby(['date', 'dock'])['MLD']
      .mean()
      .unstack('dock')
      .sort_index()
)

# ---------------------------------------------------------------------
# 2.  ANIMATION SETTINGS
# ---------------------------------------------------------------------
docks      = ['Dock_2', 'Dock_1', 'Dock_3']
avg_depths = {'Dock_1': 8.36, 'Dock_2': 7.17, 'Dock_3': 9.26}

# Compute the global maximum depth so all axes share the same y-limit
global_max = max(avg_depths.values())

frames_per_day = 30
fps            = 5

dates_ord = mean_mld.index.map(dt.date.toordinal).to_numpy()
t_interp  = np.linspace(
    dates_ord.min(),
    dates_ord.max(),
    frames_per_day * (len(dates_ord) - 1) + 1
)

dock_interp = {
    dock: np.interp(t_interp, dates_ord, mean_mld[dock].to_numpy())
    for dock in docks
}

# ---------------------------------------------------------------------
# 3.  FIGURE + GRID SPEC SETUP
# ---------------------------------------------------------------------
plt.ioff()
fig = plt.figure(figsize=(12, 7))

# Make the top row a bit larger by using height_ratios=[0.20, 0.80]
gs = fig.add_gridspec(2, 3, height_ratios=[0.20, 0.80], hspace=0.10)

# Top row for title + legend
title_ax = fig.add_subplot(gs[0, :])
title_ax.axis('off')

# Bottom row: dock panels
axes = [fig.add_subplot(gs[1, col]) for col in range(3)]

label_map = {'Dock_2': 'West', 'Dock_1': 'Central', 'Dock_3': 'East'}

# ---------------------------------------------------------------------
# 4.  INITIAL DRAWING OF RECTANGLES, LINES, TEXTS
# ---------------------------------------------------------------------
rect_top, rect_bot = [], []
ml_line, text_mld, text_lower = [], [], []

for ax, dock in zip(axes, docks):
    max_d = avg_depths[dock]

    # Use the global_max for all axes so that they share the same y-limits
    ax.set_ylim(global_max, 0)
    ax.set_xlim(0, 1)
    ax.set_xticks([])
    ax.set_ylabel('Depth (m)')
    ax.set_title(label_map[dock], fontsize=18)

    mld0 = dock_interp[dock][0]
    # Top rectangle (mixed layer)
    rt = Rectangle((0, 0), 1, mld0, facecolor='skyblue')
    # Bottom rectangle (underlying layer) only extends to this dock’s max_depth
    rb = Rectangle((0, mld0), 1, max_d - mld0, facecolor='navy', alpha=0.3)
    ax.add_patch(rt)
    ax.add_patch(rb)
    rect_top.append(rt)
    rect_bot.append(rb)

    # Horizontal line at MLD
    line_m = ax.hlines(mld0, 0, 1, colors='red', linestyles='--', linewidth=1)
    ml_line.append(line_m)

    # Text label for MLD thickness
    txt1 = ax.text(0.50, mld0 / 2, f'{mld0:.2f} m',
                   ha='center', va='center',
                   color='black', fontsize=18, fontweight='bold')
    text_mld.append(txt1)

    # Text label for underlying layer thickness
    lower_thick0 = max_d - mld0
    txt2 = ax.text(0.50, mld0 + (lower_thick0 / 2), f'{lower_thick0:.2f} m',
                   ha='center', va='center',
                   color='white', fontsize=18, fontweight='bold')
    text_lower.append(txt2)

# ---------------------------------------------------------------------
# 5.  LEGEND IN THE TOP ROW
# ---------------------------------------------------------------------

legend_elements = [
    Patch(facecolor='skyblue', label='Mixed Layer'),
    Patch(facecolor='navy', alpha=0.3, label='Underlying Layer'),
    Line2D([0], [0], color='red', linestyle='--', linewidth=2, label='MLD')
]

fig.legend(
    handles=legend_elements,
    loc='upper center',
    bbox_to_anchor=(0.5, 0.84),  # moved down slightly from 0.89
    ncol=3,
    frameon=False,
    fontsize=12)




# ---------------------------------------------------------------------
# 6.  ADD INITIAL DATE TEXT
# ---------------------------------------------------------------------
initial_date = dt.date.fromordinal(int(round(t_interp[0])))
date_text = title_ax.text(
    0.50,
    0.93,  # moved up from 0.75
    f'Daily Evolution of Mixed Layer Depth — {initial_date:%d %b %Y}',
    ha='center', va='center',
    fontsize=18, fontweight='bold'
)


# ---------------------------------------------------------------------
# 7.  UPDATE FUNCTION
# ---------------------------------------------------------------------
def update(frame_index):
    current_date = dt.date.fromordinal(int(round(t_interp[frame_index])))
    date_text.set_text(
        f'Daily Evolution of Mixed Layer Depth — {current_date:%d %b %Y}'
    )

    for j, dock in enumerate(docks):
        mld  = dock_interp[dock][frame_index]
        maxd = avg_depths[dock]
        lower_thick = maxd - mld

        rect_top[j].set_height(mld)
        rect_bot[j].set_y(mld)
        rect_bot[j].set_height(lower_thick)

        ml_line[j].set_segments([[(0, mld), (1, mld)]])

        text_mld[j].set_y(mld / 2)
        text_mld[j].set_text(f'{mld:.2f} m')

        text_lower[j].set_y(mld + (lower_thick / 2))
        text_lower[j].set_text(f'{lower_thick:.2f} m')

    artists = [date_text] + rect_top + rect_bot + ml_line + text_mld + text_lower
    return artists

# ---------------------------------------------------------------------
# 8.  ANIMATE & SAVE
# ---------------------------------------------------------------------
ani = animation.FuncAnimation(
    fig, update,
    frames=len(t_interp),
    blit=True,
    interval=1000 / fps,
    repeat=False
)

gif_path = Path('/content/drive/MyDrive/Field_Camp /correlation/mld_evolution_docks_FINAL2.gif')
ani.save(gif_path, writer='pillow', fps=fps, dpi=120)
print(f'GIF saved to: {gif_path}')

GIF saved to: /content/drive/MyDrive/Field_Camp /correlation/mld_evolution_docks_FINAL2.gif
